프롬프트 엔지니어링

# 1. 키워드 추출

## 1-1) 키워드 추출 함수

In [1]:
import openai
import os

# ✅ OpenAI API Key 설정
api_key = '' 

client = openai.OpenAI(api_key=api_key)


def get_keyword(user_input):  
    """OpenAI API를 사용하여 5개의 키워드를 추출하는 함수"""
    try:
        # 시스템 프롬프트 정의
        system_prompt = f"""
        [역할]       
        너는 사람들의 취미 영역을 듣고, 거기서 사람들이 중요하게 생각하는 스마트폰의 기능을 키워드화해서 출력해줘. 
        반드시 아래 기준을 지켜줘야 해. 

        [기준]
        1. 사용자의 취미에 중요한 스마트폰 기능/사양/부품을 추천해줘. 
        2. 5개의 키워드를 쉼표(,)로 구분해서 한 줄로만 제공해줘. 
            네가 출력하는 응답은 키워드 5개 이외의 다른 텍스트를 포함하면 절대 안 돼.
        3. 각 키워드는 한 단어고, 5글자 이내여야 해. 간결할 수록 좋아. 
        4. 키워드는 사람들이 일상에서 많이 사용하는 쉬운 단어를 선택해. 
        5. 너무 기술적이거나 긴 단어는 피해줘(예. '스타일러스', '조리개', '이퀄라이저' 제외).  
        6. 위의 사항들을 꼭 지켜서 알려줘. 

        [예시]
        사용자 입력 : '사진 촬영'
        
        원하는 결과 : '배터리', '사이즈', '화질', '용량', '초점'
        """

        # 사용자 입력 포함
        user_prompt = f'"{user_input}과 관련된 스마트폰 기능 키워드 5개를 알려줘.'

        # OpenAI API 호출
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )

        # 응답 데이터 처리
        raw_output = response.choices[0].message.content.strip()
        raw_output = raw_output.replace("'",'').split(',')
        return [res.strip() for res in raw_output]

    except Exception as e:
        print(f"오류 발생: {e}")
        return []
    

In [4]:
output = get_keyword('등산')
output

['배터리', '내비', '방수', '무게', 'GPS']

In [3]:
output = get_keyword('운동')
output

['헬스', '배터리', '위치', '시간', '앱']

In [4]:
output = get_keyword('만화책 읽기')
output

['화면', '배터리', '용량', '사이즈', '해상도']

In [6]:
import pandas as pd 
user_inputs = ['영상 제작', '사진 촬영', '디지털 아트', '음악 & 사운드', '쇼핑 & 패션', 'SNS & 커뮤니티', '여행 & 탐방', '모바일 게임', '독서 & 필기', '운동 & 건강']

res = []
cnt = 0
while cnt < 3:
    for user_input in user_inputs:
        output = get_keyword(user_input)
        print(f'''
        ▶ user_input
        {user_input}
        
        ▶ output
        {output}
        ''')
        res_output = []
        for i in output:
            res_output.append(i.split(','))
        res.append([user_input, res_output])
    cnt += 1

#pd.DataFrame(res, columns = ['user_input', 'outputs']).sort_values('user_input', ascending = False).to_csv('250221_LLM_4omini_test.csv', encoding = 'utf-8', index = False)


        ▶ user_input
        영상 제작
        
        ▶ output
        ['배터리', '화질', '용량', '렌즈', '화면']
        

        ▶ user_input
        사진 촬영
        
        ▶ output
        ['배터리', '사이즈', '해상도', '렌즈', '소음']
        

        ▶ user_input
        디지털 아트
        
        ▶ output
        ['펜슬', '화질', '화면', '배터리', '용량']
        

        ▶ user_input
        음악 & 사운드
        
        ▶ output
        ['음량', '소리', '배터리', '팬텀', '스트림']
        

        ▶ user_input
        쇼핑 & 패션
        
        ▶ output
        ['화면', '배터리', '카메라', '속도', '용량']
        

        ▶ user_input
        SNS & 커뮤니티
        
        ▶ output
        ['화면', '속도', '앱', '카메라', '배터리']
        

        ▶ user_input
        여행 & 탐방
        
        ▶ output
        ['배터리', '화질', 'GPS', '용량', '사이즈']
        

        ▶ user_input
        모바일 게임
        
        ▶ output
        ['배터리', '화면', '속도', '메모리', '소리']
        

        ▶ user_input
        독서 & 필기
        
        ▶ output
        ['화면', '저장', '배터리',

## 1-2) DB 저장

✅ 1️⃣ 실시간으로 LLM 호출 & DB 저장 후 반환 (권장)  
📌 원리:  

사용자가 웹에서 취미 입력  
DB에서 해당 취미의 키워드 검색  
있으면 바로 반환  
없으면 get_keyword() 실행 후 DB에 저장, 그리고 반환  


아래 내용 확인 필요>

MySQL 테이블 생성 (MySQL에서 실행)

In [ ]:
# 테이블 생성

USE SNS_DB;

CREATE TABLE hobbies (
    id INT AUTO_INCREMENT PRIMARY KEY, # 고유번호
    user_id INT NOT NULL, # 사용자ID
    hobby_area VARCHAR(255) NOT NULL, # 취미 영역
    keyword1 VARCHAR(100), # 키워드 1
    keyword2 VARCHAR(100), # 키워드 2
    keyword3 VARCHAR(100), # 키워드 3
    keyword4 VARCHAR(100), # 키워드 4
    keyword5 VARCHAR(100) # 키워드 5
);

# 샘플 데이터 삽입

INSERT INTO hobbies (user_id, hobby_area, keyword1, keyword2, keyword3, keyword4, keyword5)
VALUES 
(1, '사진촬영', '화질', '색감', '해상도', '밝기', '렌즈'),
(2, '등산', '화질', '배터리', '무게', '방수', 'GPS');

MySQL 서버 접속(python)

In [2]:
host_ip = "15.168.221.131"  
DATABASE = "SNS_DB"
user_id = "lab13"
user_password = "lab13"
mysql_url = f"jdbc:mysql://{host_ip}:3306/{DATABASE}"

In [5]:
import mysql.connector

# MySQL 연결 설정
conn = mysql.connector.connect(
    host='localhost',
    user='lab13',
    password='lab13',
    database='SNS_DB',
    charset='utf8mb4'
)

cursor = conn.cursor()

# 테이블 목록 확인
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()
print("📌 테이블 목록:", tables)

📌 테이블 목록: [('tbCrawled_Danawa',), ('tbCrawled_Youtube',)]


In [ ]:
# 취미 영역과 키워드 가져오기
cursor.execute("SELECT hobby_area, keyword1, keyword2, keyword3, keyword4, keyword5 FROM hobbies;")
result = cursor.fetchall()

# 결과 출력
for row in result:
    hobby_area, *keywords = row
    print(f"취미: {hobby_area}, 키워드: {', '.join(filter(None, keywords))}")



# 2. 스마트폰 추천

In [ ]:
import openai
import os

# ✅ OpenAI API Key 설정
api_key = '' 

client = openai.OpenAI(api_key=api_key)


def phone_recommend(user_input):  
    """OpenAI API를 사용하여 스마트폰을 추천하는 함수"""
    try:
        # 시스템 프롬프트 정의
        system_prompt = f"""
        [역할]       
        사람들의 취미 영역과 그 영역에서 중요한 스마트폰의 주요 기능 키워드 5개를 듣고 스마트폰 기종을 추천해줘.
        반드시 아래 기준을 지켜줘야 해. 

        [기준]
        1. 추천하는 스마트폰 기종은 3가지 종류야. 
        2. 네가 추천하는 스마트폰 기종은 현재 시점으로 단종되지 않은 기종이어야 해. 
        3. 각 기종의 기종명과 함께 유저가 입력한 기능 키워드와 관련한 기능에 대한 정보를 함께 알려줘. 
        4. 위의 사항들을 꼭 지켜서 알려줘. 

        """


    

In [ ]:
# (검토중)
        # 사용자 입력 포함
        user_prompt = f'"{user_input}과 관련된 스마트폰 기능 키워드 5개를 알려줘.'

        # OpenAI API 호출
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )

        # 응답 데이터 처리
        raw_output = response.choices[0].message.content.strip()
        raw_output = raw_output.replace("'",'').split(',')
        return [res.strip() for res in raw_output]

    except Exception as e:
        print(f"오류 발생: {e}")
        return []

In [ ]:
# 검토중
import openai

if result:
    hobby_area, *keywords = result[0]  # 첫 번째 사용자 데이터만 활용
    keyword_list = ", ".join(filter(None, keywords))

    prompt = f"""
    내가 스마트폰을 사려고 해. 
    내 취미는 '{hobby_area}'이고, 내가 중요시 하는 스마트폰의 기능은 {keyword_list}야. 
    2025년 2월 기준으로 현재 구매 가능한 스마트폰 중에서, 나에게 잘 맞을 것 같은 기종 3개를 추천해줘.
    """
else:
    prompt = "내가 스마트폰을 사려고 해. 사진 촬영이 취미고, 내가 중요시 하는 스마트폰의 기능은 화질, 색감, 해상도, 밝기, 렌즈야. 2025년 2월 기준으로 현재 구매 가능한 스마트폰 중에서, 나에게 잘 맞을 것 같은 기종 3개를 추천해줘."

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7,
    max_tokens=500
)

print(response["choices"][0]["message"]["content"])